Auteur : Michaël Leroy

reseau routier:

    * https://www.data.gouv.fr/fr/datasets/reseau-routier-dans-openstreetmap/
        - rnn2022 format shp:
            https://www.data.gouv.fr/fr/datasets/r/92d86944-52e8-44c1-b4cc-b17ac82d70ed
        - format csv: https://www.data.gouv.fr/fr/datasets/r/89f8bc85-f923-4540-bbc0-6e010b9b6339

problématique:

    * déterminer les isochrones de distance par rapport aux routes
    * agrégation des bornes contenues dans les isochrone


 A faire:
   
    * score d'équipement de la route
    * liste des communes pouvant éventuellement acceuillir une nouvelle borne avec un score de pertinence (potentiellement toutes les communes à proximité des routes grises)

# Libs

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'

# Data management
import pandas as pd
import geopandas as gpd
import numpy as np


# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.features import Choropleth
from folium.plugins import MarkerCluster

# Preprocessing
from sklearn.preprocessing import StandardScaler
import umap


# I/O
import gc
import io, requests
import zipfile, shutil
import joblib


# data_path = 'C:/Users/demo/Desktop/Lattitude/datas/'
data_path = 'datas'
os.makedirs(data_path, exist_ok=True)

# Datas

In [ ]:
# Load the datas for French charge points
file_name = 'dataset_charge_points.feather'

bornes = gpd.read_feather(os.path.join(data_path, file_name))

# Load the datas for the communes
file_name = 'dataset_communes.feather'

communes = gpd.read_feather(os.path.join(data_path, file_name))

In [ ]:
file_name_bornage = 'BORNAGE_TOUT'
file_name_vsmap = 'VSMAP_TOUT'
ext = '.shp'
url ='https://www.data.gouv.fr/fr/datasets/r/92d86944-52e8-44c1-b4cc-b17ac82d70ed'
temp_path = 'temp_unzip'


try: 
    print('Loading data from local file...')
    bornage = gpd.read_feather(os.path.join(data_path,file_name_bornage + '.feather'))
    vsmap = gpd.read_feather(os.path.join(data_path,file_name_vsmap + '.feather'))

except:
    print('Loading data from url...')
    # Zip file from url  
    zip_file = requests.get(url).content
    os.makedirs(temp_path, exist_ok=True)
    with zipfile.ZipFile(io.BytesIO(zip_file)) as archive:
        archive.extractall(temp_path)
    bornage = gpd.read_file(os.path.join(temp_path,file_name_bornage + ext))   
    vsmap = gpd.read_file(os.path.join(temp_path,file_name_vsmap + ext))   

    shutil.rmtree(temp_path) 
    
    print('Saving data to local file...')
    bornage.to_feather(os.path.join(data_path,file_name_bornage + '.feather'))
    vsmap.to_feather(os.path.join(data_path,file_name_vsmap + '.feather'))


print('Bornage :', bornage.info() )
print('Vsmap :', vsmap.info())


In [ ]:
print('Bornes crs ', bornes.crs)   
print('Vsmap crs ', vsmap.crs) 

In [ ]:
crs = 6171

bornes.to_crs(crs, inplace=True) 
# communes.to_crs(crs, inplace=True) 

vsmap.to_crs(crs, inplace=True) 
bornage.to_crs(crs, inplace=True)

In [ ]:
print('Same projection : ',bornes.crs == vsmap.crs)

In [ ]:
vsmap.plot()

In [ ]:
bornes.plot()

In [ ]:
bornes.reset_index(inplace=True)
bornes

In [ ]:
# Group routes geometries
routes = vsmap.dissolve('route')
routes

In [ ]:
# isochrones at distance from roads

distance = 40  # meters ?????????  possibly tens or hundreds

routes = vsmap.dissolve('route')

routes['buffered'] = routes.geometry.buffer(distance, cap_style=2)
routes = routes.set_geometry('buffered')

bornes['geometry_pdc'] = bornes['geometry']  # duplicates the geometry for spacial join

# Spacial join between routes buffered multipolygons and bornes points
routes_bornes = gpd.sjoin(routes,  bornes,  
                    how='inner', predicate='contains', 
                    lsuffix='_routes', rsuffix='pdc' 
                    ).reset_index()


routes_bornes.drop_duplicates(inplace=True)

# display results
# display(routes_bornes)
print(f'Found : {routes_bornes.shape[0]} bornes at distance {distance } ??? from a road.')

# Visualisation
fig, ax = plt.subplots(1,1,figsize=(10,10))

routes.set_geometry('geometry').simplify(tolerance=10).plot(color='lightgrey',ax=ax)

routes_bornes.set_geometry('geometry').simplify(tolerance=10).plot(color='blue', markersize=1, ax=ax)

routes_bornes.set_geometry('geometry_pdc').plot(color='green', ax=ax)

# communes.query("date_arrete == '2022-12-31' and PMUN > 150000").centroid.plot( markersize=30, color='red',ax=ax)

plt.show()

In [ ]:
del fig

In [ ]:
'''
icons from https://fontawesome.com/v4/icons/

'''
from folium.plugins import MarkerCluster
from folium import FeatureGroup
from shapely.geometry import Point
import math




def make_map(df_1,df_2, df_3 ):
    # Get center
    center = Point(2.1,46.3)
    
    # Create a folium map centered on the town_
    m = folium.Map(location=[center.y, center.x], zoom_start=6, crs='EPSG3857')


    # # Group 1  all french roads
    # print('Adding routes...', end=' ')
    # group_1 = folium.FeatureGroup(name='Routes')
    # for index, row in df_1.iterrows():
    #     # print(row['buffered'])
    #     # style the polygons based on "values" property
    #     # color = cmap(row.VE_per_inhab ** (1/3))
    #     def style_fn(feature):
    #         ss = {
    #             "fillColor": 'red',
    #             "fillOpacity": 0.750,
    #             "weight": 10.0,
    #             "color": 'red',
    #         }
    #         # print(row['style'], end='   ')
    #         return ss   #row['style']
    #         # folium.vector_layers.PolyLine(row.buffered,popup=row.lib_rte, tooltip=row.lib_rte
    #         #     ).add_to(group_1)           
    #         folium.GeoJson(row.geometry.buffer(20), tyle_function=style_fn
    #             ).add_to(group_1)
    #     group_1.add_to(m)
    # print('done.')    

    # Group_1 for roads isochrones
    print('Adding isochrones...', end=' ')
    group_1 = FeatureGroup(name='isochrones' )
    def style_fn(feature):
        ss = {
            "fillColor": 'lightblue',
            "fillOpacity": 0.5,
            "weight": .01,
            "color": 'lightblue',
        }
        return ss
    for index, row in df_1.iterrows():
        #communes child markers
        folium.GeoJson(row.buffered, 
                        style_function=style_fn,
                        # popup = row.lib_rte,
                        # tooltip = row.lib_rte,

                ).add_to(group_1)
    group_1.add_to(m)
    print('done.')

    # Group_2 for roads with PDC within distance
    print('Adding routes équipées...', end=' ')
    group_2 = FeatureGroup(name='Routes équipées' )
    def style_fn(feature):
        ss = {
            "fillColor": 'red',
            "fillOpacity": 0.90,
            "weight": 1.0,
            "color": 'red',
        }
        return ss
    for index, row in df_2.iterrows():
        #communes child markers
        folium.GeoJson(row['geometry'], 
                        style_function=style_fn,
                        popup = row.lib_rte,
                        tooltip = row.lib_rte,

                ).add_to(group_2)
    group_2.add_to(m)
    print('done.')


    print('Creating bornes ', end=' ')
    # Create a marker cluster layer for the data
    group_3 = MarkerCluster(name='Points de charge')
    # bornes markers
    for index, row in df_3.iterrows():
        # print(row.geometry_pdc)
        group_3.add_child(folium.Marker(location=[row.geometry_pdc.y, row.geometry_pdc.x], 
                                            # popup=popup, 
                                            tooltip='pdc infos',
                                            icon=folium.Icon(prefix='fa', 
                                            icon='bolt', 
                                            color='blue')
                    ))
    group_3.add_to(m)
    print('done.')


    # Add a layer control to the map
    folium.LayerControl().add_to(m)

    return m

crs = 4230
tolerance = 10

print('Simplify geometries... ', end=' ')
# routes_bornes['buffered'] = routes_bornes.buffered.simplify(tolerance=tolerance )
# routes_bornes['geometry'] = routes_bornes.geometry.simplify(tolerance=tolerance )
print('done.')

m = make_map(   
                routes_bornes.set_geometry('buffered').to_crs(crs),
                routes_bornes.set_geometry('geometry').to_crs(crs),
                routes_bornes.set_geometry('geometry_pdc').to_crs(crs)
)

display(m)

del m